# Optimization of spatial infraestructure for EV charging

## II. Problem formulation
$$ min \sum_{i,j} c_{ij} \pi_{ij} $$

Subject to:

$$ \sum_{j} \pi_{ij} = q_{i} $$
$$ \sum_{i} \pi_{ij} = s_{i}$$

Typical cost function:
$$C = (c_{ij})$$
Where c is a convex function, i.e.: the distance between station location and demand location in some norm:
$$c_{ij} = c(x_{i}, y_{j}) = |x_{i} - y_{j}|$$

General declarations and functions:

In [46]:
function get_cost_matrix(x, y, norm=2)    
    m = size(x)[1]
    n = size(y)[1]
    #Calculo los costos Cij = |xi-yj| (distancia euclidea en R^2)
    c1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^norm
    c2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^norm
    return (c1+c2).^(1/norm)
end

get_cost_matrix (generic function with 2 methods)

In [145]:
m = 100; # Demand locations count
n = 10; # Station locations count
norm = 2; # Manhattan Metric

In [ ]:
x = sortslices(rand(m,2), dims = 1);
y = sortslices(rand(n,2), dims = 1);
C = get_cost_matrix(x, y, norm); #Matriz de costos
q = ones(m,1);

## III. Fixed Locations
We assume that the station locations $\{ y_j \}$ have already been selected

### A. Fixed Capacities

#### Simple discrete counterexample
$$C = \begin{bmatrix}
1 & 3\\
3 & 1
\end{bmatrix}$$

$$q = \begin{bmatrix}
1\\
2
\end{bmatrix}$$

$$s = \begin{bmatrix}
2 & 1
\end{bmatrix}$$

Then:

$$\Pi^{*} = \begin{bmatrix}
1 & 0\\
1 & 1
\end{bmatrix}$$

In [25]:
using JuMP, GLPK

C = [1 3; 3 1]
q = [1; 2]
s = [2 1]


model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2, 1:2] >=0 )
@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s )

@objective(model,Min, sum( C.*pi ))

#print(model)
model

optimize!(model)
value.(pi)

2×2 Matrix{Float64}:
 1.0  0.0
 1.0  1.0

#### Proposition 1

Suppose:

* Más ubicaciones de demanda, que estaciones: $m \ge n$
* Demandas unitarias: $q_{i} = 1$, $i = 1, ..., m$
* Capacidad justa: $s_{j} \in Z, s_{j} > 0, \sum_{j} s_{j} = m$

Then: 

La solución óptima $\Pi^{*}$ debe ser de ceros y unos ($\pi_{ij} \in \{ 0, 1\}$)


In [146]:
#Vector de capacidades
aux = rand(n,1); #random proportion for s
s = round.(aux/sum(aux)*m); #normalization
s[argmax(s)] += (m - sum(s)); #error correction
#s

In [147]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [150]:
optimize!(model)
value.(pi);

In [174]:
#[TODO] Find values not in 0 or 1 to verify
find(value.(pi) != 0 || value.(pi) != 1);

LoadError: UndefVarError: find not defined

#### Case m = n

In [142]:
#Vector de capacidades
s = ones(n,1); 

In [143]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .== s' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [151]:
optimize!(model)
value.(pi);

[TODO] ¿Verificar lo de extender una $m \times n$ a $m \times m$, ponderar la capacidad y que de como el $m = n$, para luego sumar las columnas de $\Pi^{*}$ y recuperar la original ?

### B. Free Capacities

We assume that the station locations $\{ y_j \}$ have already been selected, and their supply capacities can be arbitrarly chosen.

In [171]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [172]:
optimize!(model)

10×1 adjoint(::Matrix{Float64}) with eltype Float64:
 15.0
 13.0
  7.0
 12.0
  3.0
 10.0
  6.0
 15.0
 14.0
  5.0

In [177]:
#Vector de capacidades
s = sum(value.(pi), dims=1)'

10×1 adjoint(::Matrix{Float64}) with eltype Float64:
 15.0
 13.0
  7.0
 12.0
  3.0
 10.0
  6.0
 15.0
 14.0
  5.0

### C. Constrained or penalized capacities

We assume that the station locations $\{ y_j \}$ have already been selected, and their supply capacities can be arbitrarly chosen.

[TODO] ¿Network graph?

#### Proposition 2

Suppose:

* Más ubicaciones de demanda, que estaciones: $m \ge n$
* Demandas unitarias: $q_{i} = 1$, $i = 1, ..., m$
* Capacidades topeadas (aseguran demanda): $\overline{s}_{j} \in Z, s_{j} > 0, \sum_{j} \overline{s}_{j} \ge m$

Then: 

La solución óptima $\Pi^{*}$ debe ser de ceros y unos ($\pi_{ij} \in \{ 0, 1\}$)


In [ ]:
#Vector de cotas de capacidad
s_max = 

In [ ]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi, dims=2) .== q)
@constraint(model, sum(pi, dims=1) .<= s_max' )

@objective(model,Min, sum( C.*pi ))

#print(model)
model;

In [ ]:
optimize!(model)

In [ ]:
value.(pi);

In [8]:
smax = 100*ones(n,1) # Capacidad máxima de cargadores
k = 3 # Cantidad de estaciones prendidas
q = 1*ones(m,1); # Demandas

In [9]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)
@variable(model, b[1:n], Bin)

@constraint(model, sum(pi, dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= (b.*smax) )
@constraint(model, sum(b) <= k)

@objective(model,Min, sum( c.*pi ))

#print(model)
model

A JuMP Model
Minimization problem with:
Variables: 1010
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 100 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 11 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: b, pi

In [10]:
optimize!(model)
value.(pi), objective_value(model)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 1.0 0.0], 27.278353011912728)

In [11]:
value.(pi)

100×10 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0

In [12]:
value.(b)

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0

### Fixed capacities

In [11]:
c = [1 3; 3 1] # Costo
s = [0 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)

In [12]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2,1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .== s )

@objective(model,Min, sum( c.*pi ))

print(model)

optimize!(model)
value.(pi), objective_value(model)

([0.0 1.0; 0.0 -1.0], 2.0)

In [13]:
value.(pi)

2×2 Matrix{Float64}:
 0.0   1.0
 0.0  -1.0

## Constrained or penalized capacities

In [14]:
c = [1 3; 3 1] # Costo
smax = [2 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)

In [15]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2,1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= smax )

@objective(model,Min, sum( c.*pi ))

print(model)

optimize!(model)
value.(pi), objective_value(model)

([1.0 0.0; 0.0 1.0], 2.0)

In [16]:
value.(pi)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

### Penalized

In [17]:
function phi(s)
    return 0.5*(s.^2)
end

phi (generic function with 1 method)

In [18]:
c = [1 3; 3 1] # Costo
smax = [2 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)
eps = 0.1;

In [19]:
#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)

@variable(model, pi[1:2,1:2]>=0)
@variable(model, s[1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .== s )

@objective(model,Min, sum( c.*pi ) + eps*sum(phi(s)))

print(model)

optimize!(model)
value.(pi), objective_value(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality co

([1.0000000087461995 -8.746199604432545e-9; -8.746199604432545e-9 1.0000000087461995], 2.099999965015201)

In [20]:
value.(pi)

2×2 Matrix{Float64}:
  1.0        -8.7462e-9
 -8.7462e-9   1.0